<a href="https://colab.research.google.com/github/RashmikaReddy/Named-Entity-Recognition/blob/main/NER_BERT_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from numpy import NaN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 182 kB 78.7 MB/s 
     |████████████████████████████████| 7.6 MB 67.3 MB/s 


In [ ]:
import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/EbayChallenge/Train_Tagged_Titles.tsv', sep="\t",on_bad_lines='skip')

In [ ]:
class DataSequenceTest(torch.utils.data.Dataset):

    def __init__(self, df):
        #print("stage 2\n")
        txt = Data_processed['Title'].values.tolist()
        #print("stage 3\n")
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]

    def get_batch_data(self, idx):

        return self.texts[idx]

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)

        return batch_data

In [ ]:
data.head(20)

,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,NaN
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model
5,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Multi-color,Color
6,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,canvas,Fabric Type
7,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Multi-color,Color
8,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,canvas,Fabric Type
9,2,LOUIS VUITTON Petit Noe Drawstring Shoulder Ba...,LOUIS,Brand


In [ ]:
data['Tag'].unique()

array(['Brand', nan, 'MPN', 'Type', 'Model', 'Color', 'Fabric Type',
       'Closure', 'Pattern', 'Material', 'No Tag', 'Product Line',
       'Features', 'Theme', 'Country/Region of Manufacture', 'Occasion',
       'Department', 'Size', 'Handle/Strap Material', 'Handle Style',
       'Accents', 'Style', 'Hardware Material', 'Trim Material',
       'Measurement, dimension', 'Pocket Type', 'Character', 'Obscure',
       'Season', 'Character Family', 'Lining Material', 'Strap Drop',
       'Handle Drop'], dtype=object)

In [ ]:
#Number of texts we have are 4997
data['Record Number'].nunique()

4997

In [ ]:
len(data)

55080

In [ ]:
data_df = data[['Record Number','Token','Tag']]

In [ ]:
data_df = data_df.rename(columns={"Record Number": "sentence_id", "Token": "words","Tag":"labels"})


In [ ]:
data_df

,sentence_id,words,labels
0,1,LOUIS,Brand
1,1,VUITTON,NaN
2,1,M40096,MPN
3,1,Handbag,Type
4,1,Priscilla,Model
...,...,...,...
55075,5000,Top,No Tag
55076,5000,Closure,No Tag
55077,5000,Retail,No Tag
55078,5000,$,No Tag


In [ ]:
data_df.to_csv('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/data_bio_shuffled_total.csv')



In [ ]:
dictionary = {}
prev_rownumber = 1
prev_token = ""
for index,row in data.iterrows():
  rownumber = row['Record Number']
  token = row['Token']
  if rownumber != prev_rownumber:
      prev_rownumber = rownumber
  if rownumber == prev_rownumber:
    if row['Tag'] == 'No Tag':
      s = s+";"+"O"
    elif row['Tag'] is not NaN:
      s = s+";"+"B-"+row['Tag']
      prev_tag = row['Tag']
    elif row['Tag'] is NaN:
      s = s+";"+"I-"+prev_tag
    prev_rownumber = rownumber
    prev_token = token

In [ ]:
dictionary = {}
unique_tags = set()
prev_rownumber = 1
s = ""
prev_title = ""
for index,row in data.iterrows():
  title = row['Title']
  rownumber = row['Record Number']
  if rownumber != prev_rownumber:
    if len(s) != 0:
      dictionary[prev_title] = s[1:]
      s = ""
      prev_rownumber = rownumber
  if rownumber == prev_rownumber:
    if row['Tag'] == 'No Tag':
      s = s+";"+"O"
      unique_tags.add("O")
    elif row['Tag'] is not NaN:
      s = s+";"+"B-"+row['Tag']
      unique_tags.add("B-"+row['Tag'])
      prev_tag = row['Tag']
    elif row['Tag'] is NaN:
      s = s+";"+"I-"+prev_tag
      unique_tags.add("I-"+prev_tag)
    prev_rownumber = rownumber
    prev_title = title
dictionary[prev_title] = (s.lstrip()).rstrip()

  
  




In [ ]:
dictionary

{'LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas': 'B-Brand;I-Brand;B-MPN;B-Type;B-Model;B-Color;B-Fabric Type;B-Color;B-Fabric Type',
 'LOUIS VUITTON Petit Noe Drawstring Shoulder Bag Monogram Leather M42226 39SD442': 'B-Brand;I-Brand;B-Model;I-Model;B-Closure;B-Type;B-Type;B-Pattern;B-Material;B-MPN;O',
 'LOUIS VUITTON Damier Azur Pochette Bosphore Shoulder Bag N51112 LV Auth yt523': 'B-Brand;I-Brand;B-Product Line;B-Color;B-Product Line;B-Model;B-Type;B-Type;B-MPN;B-Brand;O;O',
 'GUCCI Bamboo 2Way Shoulder Bag Leather Brown Auth fm1002': 'B-Brand;B-Product Line;B-Features;B-Type;B-Type;B-Material;B-Color;O;B-MPN',
 'Rank AB Vintage Gucci Sherry line PVC Leather Clutch bag Brown From Japan A128': 'O;O;O;B-Brand;B-Product Line;I-Product Line;B-Material;I-Material;B-Type;B-Type;B-Color;O;O;O',
 '1970s NYC Bonnie Cashin Coach Brown Gray Leather Saddle Pouch Crossbody Bag': 'B-Theme;B-Country/Region of Manufacture;B-Model;I-Model;B-Brand;B-Color;B-Color;B-Ma

In [ ]:
len(dictionary)

NameError: ignored

In [ ]:
Data_processed = pd.DataFrame(list(dictionary.items()),columns = ['Title','Tag'])

In [ ]:
Data_processed.head(10)

,Title,Tag
0,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,B-Brand;I-Brand;B-MPN;B-Type;B-Model;B-Color;B...
1,LOUIS VUITTON Petit Noe Drawstring Shoulder Ba...,B-Brand;I-Brand;B-Model;I-Model;B-Closure;B-Ty...
2,LOUIS VUITTON Damier Azur Pochette Bosphore Sh...,B-Brand;I-Brand;B-Product Line;B-Color;B-Produ...
3,GUCCI Bamboo 2Way Shoulder Bag Leather Brown A...,B-Brand;B-Product Line;B-Features;B-Type;B-Typ...
4,Rank AB Vintage Gucci Sherry line PVC Leather ...,O;O;O;B-Brand;B-Product Line;I-Product Line;B-...
5,1970s NYC Bonnie Cashin Coach Brown Gray Leath...,B-Theme;B-Country/Region of Manufacture;B-Mode...
6,LOUIS VUITTON Epi Serviette Fermoir Business B...,B-Brand;I-Brand;B-Material;B-Model;I-Model;B-O...
7,Womens Handbag,B-Department;B-Type
8,Chanel Leo Lion Flap Bag Chevron Lambskin Medium,B-Brand;B-Model;I-Model;B-Type;B-Type;B-Patter...
9,NEW FOSSIL GREEN PLAID CANVAS ESTATE CVS BRIEF...,O;B-Brand;B-Color;B-Pattern;B-Fabric Type;B-Mo...


In [ ]:
tags_to_ids = {k: v for v, k in enumerate(sorted(unique_tags))}
ids_to_tags = {v: k for v, k in enumerate(sorted(unique_tags))}

In [ ]:
print(sorted(unique_tags))

['B-Accents', 'B-Brand', 'B-Character', 'B-Character Family', 'B-Closure', 'B-Color', 'B-Country/Region of Manufacture', 'B-Department', 'B-Fabric Type', 'B-Features', 'B-Handle Drop', 'B-Handle Style', 'B-Handle/Strap Material', 'B-Hardware Material', 'B-Lining Material', 'B-MPN', 'B-Material', 'B-Measurement, dimension', 'B-Model', 'B-Obscure', 'B-Occasion', 'B-Pattern', 'B-Pocket Type', 'B-Product Line', 'B-Season', 'B-Size', 'B-Strap Drop', 'B-Style', 'B-Theme', 'B-Trim Material', 'B-Type', 'I-Accents', 'I-Brand', 'I-Character', 'I-Character Family', 'I-Closure', 'I-Color', 'I-Country/Region of Manufacture', 'I-Department', 'I-Fabric Type', 'I-Features', 'I-Handle Drop', 'I-Handle Style', 'I-Handle/Strap Material', 'I-Hardware Material', 'I-MPN', 'I-Material', 'I-Measurement, dimension', 'I-Model', 'I-Occasion', 'I-Pattern', 'I-Pocket Type', 'I-Product Line', 'I-Size', 'I-Strap Drop', 'I-Style', 'I-Theme', 'I-Trim Material', 'I-Type', 'O']


In [ ]:
tags_to_ids

{'B-Accents': 0,
 'B-Brand': 1,
 'B-Character': 2,
 'B-Character Family': 3,
 'B-Closure': 4,
 'B-Color': 5,
 'B-Country/Region of Manufacture': 6,
 'B-Department': 7,
 'B-Fabric Type': 8,
 'B-Features': 9,
 'B-Handle Drop': 10,
 'B-Handle Style': 11,
 'B-Handle/Strap Material': 12,
 'B-Hardware Material': 13,
 'B-Lining Material': 14,
 'B-MPN': 15,
 'B-Material': 16,
 'B-Measurement, dimension': 17,
 'B-Model': 18,
 'B-Obscure': 19,
 'B-Occasion': 20,
 'B-Pattern': 21,
 'B-Pocket Type': 22,
 'B-Product Line': 23,
 'B-Season': 24,
 'B-Size': 25,
 'B-Strap Drop': 26,
 'B-Style': 27,
 'B-Theme': 28,
 'B-Trim Material': 29,
 'B-Type': 30,
 'I-Accents': 31,
 'I-Brand': 32,
 'I-Character': 33,
 'I-Character Family': 34,
 'I-Closure': 35,
 'I-Color': 36,
 'I-Country/Region of Manufacture': 37,
 'I-Department': 38,
 'I-Fabric Type': 39,
 'I-Features': 40,
 'I-Handle Drop': 41,
 'I-Handle Style': 42,
 'I-Handle/Strap Material': 43,
 'I-Hardware Material': 44,
 'I-MPN': 45,
 'I-Material': 46,
 

In [ ]:
data['Tag'].unique()

array(['Brand', nan, 'MPN', 'Type', 'Model', 'Color', 'Fabric Type',
       'Closure', 'Pattern', 'Material', 'No Tag', 'Product Line',
       'Features', 'Theme', 'Country/Region of Manufacture', 'Occasion',
       'Department', 'Size', 'Handle/Strap Material', 'Handle Style',
       'Accents', 'Style', 'Hardware Material', 'Trim Material',
       'Measurement, dimension', 'Pocket Type', 'Character', 'Obscure',
       'Season', 'Character Family', 'Lining Material', 'Strap Drop',
       'Handle Drop'], dtype=object)

Pre processing part 2

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
text_tokenized = tokenizer(Data_processed['Title'][0], padding='max_length', max_length=30, truncation=True)
print(text_tokenized)





{'input_ids': [101, 149, 2346, 22054, 1708, 159, 22054, 20174, 11414, 150, 12882, 1568, 1580, 1545, 9918, 17097, 153, 4889, 23694, 18447, 21829, 11246, 18447, 21829, 11246, 102, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]}


As sub-word tokenization, word-piece tokenization splits uncommon words into their sub-words, such as ‘Geir’ and ‘Haarde’ in the example above. This sub-word tokenization helps the BERT model to learn the semantic meaning of related words.

The consequence of this word piece tokenization and the addition of special tokens from BERT is that the sequence length after tokenization is no longer matching the length of the initial label.

From the example above, now there are in total 30 tokens in the sequence after tokenization, while the length of the label is still the same as before. Also, the first token in a sequence is no longer the word 'LOUIS', but the newly added [CLS] token, so we need to shift our label as well.

In [ ]:
#word_ids is the output that basically tells us how the words are divided into subwords, what are the padding, CLS, SEP words are as they are represented as None,
#word_id 0 is the word Louis which is divided into 4 subwords 
print(Data_processed['Title'][0])
text_tokenized.word_ids()

LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas


[None,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 4,
 4,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 None,
 None,
 None]

In [ ]:
Data_processed['Title'][0]
Data_processed['Tag'][0]

'B-Brand;I-Brand;B-MPN;B-Type;B-Model;B-Color;B-Fabric Type;B-Color;B-Fabric Type'

In [ ]:
lb = [i.split(';') for i in Data_processed['Tag'].values.tolist()]
lb[0:2]

[['B-Brand',
  'I-Brand',
  'B-MPN',
  'B-Type',
  'B-Model',
  'B-Color',
  'B-Fabric Type',
  'B-Color',
  'B-Fabric Type'],
 ['B-Brand',
  'I-Brand',
  'B-Model',
  'I-Model',
  'B-Closure',
  'B-Type',
  'B-Type',
  'B-Pattern',
  'B-Material',
  'B-MPN',
  'O']]

In [ ]:
txt = Data_processed['Title'].values.tolist()
txt[0:2]


['LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas',
 'LOUIS VUITTON Petit Noe Drawstring Shoulder Bag Monogram Leather M42226 39SD442']

In [ ]:
#removing any semi colons in the starting
Data_processed['Tag'] = Data_processed['Tag'].str.lstrip(';')

In [ ]:
import string
for index,row in Data_processed.iterrows():
    #row['Title'] = row['Title'].lower()
    row['Title'] = row['Title'].replace('-', '')
    row['Title'] = row['Title'].replace('+', '')
    row['Title'] = row['Title'].replace("'s", '')
    row['Title'] = row['Title'].replace("#", '')
    row['Title'] = row['Title'].replace("/", '')
    row['Title'] = row['Title'].replace("*", '')
    row['Title'] = row['Title'].replace(",", '')
    row['Title'] = row['Title'].replace("/", '')
    row['Title'] = row['Title'].replace("!", '')
    row['Title'] = row['Title'].replace(".", '')
    row['Title'] = row['Title'].replace('‑', '')
    row['Title'] = row['Title'].replace(':', '')
    row['Title'] = row['Title'].replace('(', '')
    row['Title'] = row['Title'].replace(')', '')
    row['Title'] = row['Title'].replace('[', '')
    row['Title'] = row['Title'].replace(']', '')
    row['Title'] = row['Title'].replace(';', '')
    row['Title'] = row['Title'].replace('?', '')
    row['Title'] = row['Title'].replace('  ', ' ')
    row['Title'] = row['Title'].replace('芬迪', '')
    row['Title'] = row['Title'].replace("’", '')

    row['Title'] = row['Title'].translate(str.maketrans('', '', string.punctuation))






 








Data_processed['Title'][0]
#print("LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas".replace('-',''))

'LOUIS VUITTON M40096 Handbag Priscilla Multicolor canvas Multicolor canvas'

In [ ]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
        #print("word_idx is", word_idx)
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx == previous_word_idx:
            label_ids.append(-100)
        else:
            #print("labels[word_idx]", labels[word_idx])
            #print("tags_to_ids",tags_to_ids[labels[word_idx]])
            label_ids.append(tags_to_ids[labels[word_idx]])
        previous_word_idx = word_idx

    return label_ids

In [ ]:
#lb = [i.split(';') for i in Data_processed['Tag'].values.tolist()]
print(Data_processed['Tag'][4996])


B-Brand;B-Model;B-Size;B-Type;B-Type;B-Color;B-Material;B-Closure;O;O;O;O;O


In [ ]:
lb = [i.split(';') for i in Data_processed['Tag'].values.tolist()]
        #print("stage 2\n")
txt = Data_processed['Title'].values.tolist()
        #print("stage 3\n")
texts = [tokenizer(str(i),padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
count = 0
for i,j in zip(txt, lb):
  count = count + 1
  #print("txt is",i)
  #print("label is",j)
  #print(count)
  #txt = 'LOUIS VUITTON M40096 Handbag Priscilla Multicolor canvas Multicolor canvas'
  labels = [align_label(i,j) for i,j in zip(txt, lb)]
print(labels)

KeyboardInterrupt: ignored

In [ ]:
import torch

label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
        #print("word_idx is", word_idx)
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx == previous_word_idx:
            label_ids.append(-100)
        else:
            #print("labels[word_idx]", labels[word_idx])
            #print("tags_to_ids",tags_to_ids[labels[word_idx]])
            label_ids.append(tags_to_ids[labels[word_idx]])
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):
        #print("stage 1\n")
        lb = [i.split(';') for i in Data_processed['Tag'].values.tolist()]
        #print("stage 2\n")
        txt = Data_processed['Title'].values.tolist()
        #print("stage 3\n")
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [ ]:
df_train, df_val, df_test = np.split(Data_processed.sample(frac=1, random_state=42),
                            [int(.8 * len(Data_processed)), int(.9 * len(Data_processed))])

In [ ]:
df_train.shape

(3997, 2)

In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(3997, 2)
(500, 2)
(500, 2)


In [ ]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(unique_tags))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [ ]:
def train_loop(model, df_train, df_val):
    #print("stage 0\n")
    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=2, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=2, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)
            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]
              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0
        total_values = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)
            #print("loss is", loss)
            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]
              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()
              total_values = total_values + 1

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / total_values: .3f} | Accuracy: {total_acc_val / total_values: .3f}')

LEARNING_RATE = 5e-3
EPOCHS = 4
BATCH_SIZE = 2

model = BertModel()
train_loop(model, df_train, df_val)
torch.save(model,'BERT.pkl')

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Epochs: 1 | Loss:  1.477 | Accuracy:  0.751


100%|██████████| 2499/2499 [08:21<00:00,  4.98it/s]


Epochs: 2 | Loss:  1.021 | Accuracy:  0.782


100%|██████████| 2499/2499 [08:21<00:00,  4.98it/s]


Epochs: 3 | Loss:  0.847 | Accuracy:  0.828


100%|██████████| 2499/2499 [08:22<00:00,  4.98it/s]


Epochs: 4 | Loss:  0.728 | Accuracy:  0.860


In [ ]:
torch.save(model,'/content/drive/MyDrive/EbayChallenge/BERT_new.pkl')

In [ ]:
model2 = torch.load('/content/drive/MyDrive/EbayChallenge/BERT.pkl')

In [ ]:

def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0
    total_length = 0


    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][test_label[i] != -100]
              label_clean = test_label[i][test_label[i] != -100]
              

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_length = total_length + 1
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / total_length: .3f}')


evaluate(model2, df_test)

Test Accuracy:  0.860
